# Opening llc4320 data and grid from the Open Storage Network

## C Spencer Jones (Texas A&M University), Chris Hill (Massachusetts Institute of Technology) and Thomas Haine (Johns Hopkins University)

Eta, U, V, W, T and S at the surface are available: these variables are grouped together with one file for each face and time. 

KPPhbl, PhiBot, oceTAUX, oceTAUY and SIarea: these variables are grouped together with one file for each face and time.

More fields may be available in the future. 

In [1]:
import s3fs
import xarray as xr
import ujson
import dask
import numpy as np
from xmitgcm import llcreader

from xarray.core.combine import _nested_combine
from functools import partial


def _multi_file_closer(closers):
    for closer in closers:
        closer()

First, I need you to decide how much of the data you want to use. I do not recommend loading all of the metadata at once, because that involves loading a huge number of json files. It will take a long time and use quite a lot of RAM. Experiment on your own machine (and with your own connection). 

In [2]:
# You (the user) should edit this for your use case. 
# Define which face or faces you are using
face_range = range(10,11)
#possible values are from 0 to 12

#define which iterations you want. 
start_from = 1180 # 0 is the first record of Eta, ~1180 is the first record for wind
length_in_hours = 12
time_step_in_hours = 3 # the minimum timestep between files is one hour


start_iter = 10368 + start_from*144
end_iter = 10368 + start_from*144 + length_in_hours*144
iter_range = np.arange(start_iter,end_iter,time_step_in_hours*144)

get_Eta_files = True
#Note that some of the early times do not have wind data uploaded. 

In [3]:
#code to create a lazily loaded xarray dataset with pointers to the data on OSN

endpoint_url = 'https://mghp.osn.xsede.org'
fs = s3fs.S3FileSystem(anon=True,
    client_kwargs={'endpoint_url': endpoint_url}
)


if (get_Eta_files):
    filelist = ['cnh-bucket-1/llc_surf/kerchunk_files/llc4320_Eta-U-V-W-Theta-Salt_f' + str(var1) + '_k0_iter_' + 
            str(var) + '.json' for var1 in face_range for var in iter_range]
else:
    filelist = ['cnh-bucket-1/llc_wind/kerchunk_files/llc4320_KPPhbl-PhiBot-oceTAUX-oceTAUY-SIarea_f' + str(var1) + '_k0_iter_' + 
            str(var) + '.json' for var1 in face_range for var in iter_range]


mapper = [fs.open(file, mode='rb') for file in filelist]


reflist = [ujson.load(mapper1)for mapper1 in mapper]

open_ = dask.delayed(xr.open_dataset)
getattr_ = dask.delayed(getattr)

dict_list = [{"storage_options": {"fo": p,"remote_protocol": "s3",
    "remote_options": {"client_kwargs":{'endpoint_url': endpoint_url}, "anon": True}},"consolidated": False} for p in reflist]
datasets = [open_("reference://",engine="zarr",backend_kwargs=p,chunks={'i':720, 'j':720}) for p in dict_list]
closers = [getattr_(ds, "_close") for ds in datasets]
datasets, closers = dask.compute(datasets, closers)


ds = xr.combine_by_coords([dataset for dataset in datasets],compat="override", coords='minimal', combine_attrs='override')


for ds1 in datasets:
            ds1.close()

ds.set_close(partial(_multi_file_closer, closers))

In [4]:
#Here's your dataset! Remember it's not loaded, and the data is still out there on open storage network. 
#If you load, the data will be downloaded to your local machine
ds

<xarray.Dataset> Size: 2GB
Dimensions:  (time: 4, face: 1, j: 4320, i: 4320, k: 1, i_g: 4320, j_g: 4320,
              k_l: 1, k_p1: 2, k_u: 1)
Coordinates:
  * face     (face) float64 8B 10.0
  * i        (i) float64 35kB 0.0 1.0 2.0 3.0 ... 4.317e+03 4.318e+03 4.319e+03
  * i_g      (i_g) float64 35kB 0.0 1.0 2.0 ... 4.317e+03 4.318e+03 4.319e+03
  * j        (j) float64 35kB 0.0 1.0 2.0 3.0 ... 4.317e+03 4.318e+03 4.319e+03
  * j_g      (j_g) float64 35kB 0.0 1.0 2.0 ... 4.317e+03 4.318e+03 4.319e+03
  * k        (k) float64 8B 0.0
  * k_l      (k_l) float64 8B 0.0
  * k_p1     (k_p1) float64 16B 0.0 1.0
  * k_u      (k_u) float64 8B 0.0
    niter    (time) float64 32B dask.array<chunksize=(1,), meta=np.ndarray>
  * time     (time) datetime64[ns] 32B 2011-11-01T04:00:00 ... 2011-11-01T13:...
Data variables:
    Eta      (time, face, j, i) float32 299MB dask.array<chunksize=(1, 1, 720, 720), meta=np.ndarray>
    Salt     (time, k, face, j, i) float32 299MB dask.array<chunksize=(1, 1, 1, 720, 720), meta=np.ndarray>
    Theta    (time, k, face, j, i) float32 299MB dask.array<chunksize=(1, 1, 1, 720, 720), meta=np.ndarray>
    U        (time, k, face, j, i_g) float32 299MB dask.array<chunksize=(1, 1, 1, 720, 360), meta=np.ndarray>
    V        (time, k, face, j_g, i) float32 299MB dask.array<chunksize=(1, 1, 1, 360, 720), meta=np.ndarray>
    W        (time, k_l, face, j, i) float32 299MB dask.array<chunksize=(1, 1, 1, 720, 720), meta=np.ndarray>
Attributes:
    history:  Created by C. Spencer Jones using https://github.com/cspencerjo...
    source:   MITgcm
    title:    Chunked netcdf created from MITgcm LLC4320 surface output

The dataset is more useful if you have the grid as well: here is how to get that. 

In [5]:
#code to create a lazily loaded xarray dataset of the llc4320 grid with pointers to the data on OSN

endpoint_url = 'https://mghp.osn.xsede.org'
fs = s3fs.S3FileSystem(anon=True,
    client_kwargs={'endpoint_url': endpoint_url}
)


filelist = ['cnh-bucket-1/llc_surf/kerchunk_files/llc4320_grid_f' + str(var1) + '.json' for var1 in range(0,13)]


mapper = [fs.open(file, mode='rb') for file in filelist]


reflist = [ujson.load(mapper1)for mapper1 in mapper]

open_ = dask.delayed(xr.open_dataset)
getattr_ = dask.delayed(getattr)

dict_list = [{"storage_options": {"fo": p,"remote_protocol": "s3",
    "remote_options": {"client_kwargs":{'endpoint_url': endpoint_url}, "anon": True}},"consolidated": False} for p in reflist]
datasets = [open_("reference://",engine="zarr",backend_kwargs=p,chunks={}) for p in dict_list]
closers = [getattr_(ds, "_close") for ds in datasets]
datasets, closers = dask.compute(datasets, closers)


co = xr.combine_by_coords([dataset for dataset in datasets],compat="override", coords='minimal', combine_attrs='override')


for ds1 in datasets:
            ds1.close()

co.set_close(partial(_multi_file_closer, closers))

In [6]:
co

<xarray.Dataset> Size: 17GB
Dimensions:  (face: 13, j: 4320, i: 4320, j_g: 4320, i_g: 4320, time: 10312)
Coordinates: (12/38)
    CS       (face, j, i) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    Depth    (face, j, i) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    PHrefC   float32 4B ...
    PHrefF   float32 4B ...
    SN       (face, j, i) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    XC       (face, j, i) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    ...       ...
    rA       (face, j, i) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    rAs      (face, j_g, i) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    rAw      (face, j, i_g) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    rAz      (face, j_g, i_g) float32 970MB dask.array<chunksize=(1, 360, 360), meta=np.ndarray>
    rhoRef   float32 4B ...
  * time     (time) datetime64[ns] 82kB 2011-09-13 ... 2012-11-15T15:00:00
Data variables:
    *empty*

CSJ acknowledges funding from NASA’s SWOT Science Team program, grant number 80NSSC20K1142. The data provided here is hosted on a pod in XSEDE's Open Storage Network, and may disappear if the underlying hardware fails: we will make every effort to keep this data online, but we cannot guarantee continued access to this data. We thanks XSEDE for providing this storage. 